In [1]:
from sedona.spark import *

config = (
    SedonaContext.builder()
    .config(
        "spark.jars.packages",
        "org.apache.sedona:sedona-spark-3.0_2.12:1.5.1,"
        "org.datasyslab:geotools-wrapper:1.5.1-28.2",
    )
    .config(
        "spark.jars.repositories",
        "https://artifacts.unidata.ucar.edu/repository/unidata-all",
    )
    .getOrCreate()
)

sedona = SedonaContext.create(config)

Skipping SedonaKepler import, verify if keplergl is installed
Skipping SedonaPyDeck import, verify if pydeck is installed


https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/anthony/.ivy2/cache
The jars for the packages stored in: /home/anthony/.ivy2/jars


:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


org.apache.sedona#sedona-spark-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-85e2b95d-1340-425b-90e2-c3c1df54b76f;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-3.0_2.12;1.5.1 in central
	found org.apache.sedona#sedona-common;1.5.1 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.19.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	fo

In [2]:
! gcloud storage du --readable-sizes gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010

1.15kiB      gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/Readme.txt
11.08MiB     gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio1.tif
11.86MiB     gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio10.tif
10.99MiB     gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio11.tif
67.06MiB     gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio12.tif
41.57MiB     gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio13.tif
23.64MiB     gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio14.tif
20.99MiB     gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio15.tif
55.30MiB     gs:

In [3]:
# Caused by: org.apache.spark.SparkException: The length of gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Elevation/ASTER_Elevation.tif is 13198053670, which exceeds the max length allowed: 2147483647.
! gcloud storage du --readable-sizes gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Elevation/ASTER_Elevation.tif

12.29GiB     gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Elevation/ASTER_Elevation.tif


In [4]:
from pyspark.sql import functions as F

base_path = "gs://dsgt-clef-geolifeclef-2024/data/raw"
df = sedona.read.format("binaryFile").load(
    # can replace bio10 with * to load all files
    f"{base_path}/EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio10.tif"
)
df = df.withColumn("raster", F.expr("RS_FromGeoTiff(content)"))
df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- raster: raster (nullable = true)



In [6]:
%time df.show(n=1, vertical=True, truncate=100)

-RECORD 0----------------------------------------------------------------------------------------------------------------
 path             | gs://dsgt-clef-geolifeclef-2024/data/raw/EnvironmentalRasters/Climate/BioClimatic_Average_1981-20... 
 modificationTime | 2024-02-08 00:08:24.479                                                                              
 length           | 12434023                                                                                             
 content          | [49 49 2A 00 08 00 00 00 12 00 00 01 03 00 01 00 00 00 CF 1F 00 00 01 01 03 00 01 00 00 00 5B 15 ... 
 raster           | GridCoverage2D["geotiff_coverage", GeneralEnvelope[(-32.266806146449994, 26.633193977950008), (35... 

CPU times: user 9.19 ms, sys: 251 µs, total: 9.44 ms
Wall time: 22.9 s
